# Code Optimization [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ua-2025q3-astr501-513/ua-2025q3-astr501-513.github.io/blob/main/501/05/lab.ipynb)

Speeding Up Your Python Codes 1000x!!!

## Introduction

Welcome to "Code Optimization: Speeding Up Your Python Codes 1000x"!

In the next hours, we will learn tricks to unlock python performance
and increase a sample code's speed by a factor of 1000x!
This lab is based on a Hack Arizona tutorial CK gave in 2024.
The [original lab](https://github.com/rndsrc/orbits-py) can be found
here.

Here is our plan:
* Understand the problem:
  we will start by outlining the $n$-body problem and how to use the
  leapfrog algorithm to solve it numerically.
* Benchmark and profile:
  we will learn how to measure performance and identify bottlenecks
  using tools like timeit and cProfile.
* Optimize the code:
  step through a series of optimizations:
  * Use list comprehensions and reduce operation counts.
  * Replace slow operations with faster ones.
  * Use high-performance libraries such as NumPy.
  * Explore lower precision where applicable.
  * Use Google JAX for just-in-time compilation and GPU acceleration.
* Apply the integrator:
  finally, we will use our optimized code to simulate the mesmerizing
  figure-8 self-gravitating orbit.

By the end of this lab, you will see that Python isn't just great for
rapid prototyping.
It can also deliver serious performance when optimized right.
Let's dive in!

## The $n$-body Problem and the Leapfrog Algorithm

The $n$-body problem involves simulating the motion of many bodies
interacting with each other through (usually) gravitational forces.

This problem is physically interesting, as it models the
[solar system](https://rebound.readthedocs.io/en/latest/),
[galaxy dynamics, or even the whole cosmic structure
formation](https://wwwmpa.mpa-garching.mpg.de/gadget/)!

This problem is also computational interesting.
Each body experiences a force from every other body, making the
simulation computationally intensive.
It is ideal for exploring optimization techniques.

The leapfrog algorithm is a simple, robust numerical integrator that
alternates between updating velocities ("kicks") and positions
("drifts").
It is popular because it conserves energy and momentum well over long
simulation periods.

We need to solve Newton's second law of motion for a collection of $n$
bodies.
The body forces are solved by using Newton's Law of Universal
Gravitation.

### Gravitational Force/Acceleration Equation

For any two bodies labeled by $i$ and $j$, the direct gravitational
force exerted on body $i$ by body $j$ is given by:
\begin{align}
  \mathbf{f}_{ij} = -G m_i m_j\frac{\mathbf{r}_i - \mathbf{r}_j\ \ \ }{|\mathbf{r}_i - \mathbf{r}_j|^{3/2}},
\end{align}
where:
* $G$ is the gravitational constant.
* $m_i$ and $m_j$ are the masses of the bodies.
* $\mathbf{r}_i$ and $\mathbf{r}_j$ are their position vectors.

Summing the contributions from all other bodies gives the net force
(and thus the acceleration) on body $i$:
\begin{align}
  \mathbf{f}_i
  = \sum_{j\ne i} \mathbf{f}_{ij}
  = -G m_i \sum_{j\ne i} m_j \frac{\mathbf{r}_i - \mathbf{r}_j\ \ \ }{|\mathbf{r}_i - \mathbf{r}_j|^3},
\end{align}

Given Newton's law $\mathbf{f} = m\mathbf{a}$, the "acceleration"
applied on body $i$ caused by all other bodies is:
\begin{align}
  \mathbf{a}_i
  = \sum_{j\ne i} \mathbf{a}_{ij}
  = -G \sum_{j\ne i} m_j \frac{\mathbf{r}_i - \mathbf{r}_j\ \ \ }{|\mathbf{r}_i - \mathbf{r}_j|^3}.
\end{align}
Choosing the right units so $G = 1$.

Here is a pure Python implementation of the gravitational
acceleration:

In [ ]:
def acc1(m, r):
    
    n = len(m)
    a = []
    for i in range(n):
        axi, ayi, azi = 0, 0, 0
        for j in range(n):
            if j != i:
                xi, yi, zi = r[i]
                xj, yj, zj = r[j]

                axij = - m[j] * (xi - xj) / ((xi - xj)**2 + (yi - yj)**2 + (zi - zj)**2)**(3/2)
                ayij = - m[j] * (yi - yj) / ((xi - xj)**2 + (yi - yj)**2 + (zi - zj)**2)**(3/2)
                azij = - m[j] * (zi - zj) / ((xi - xj)**2 + (yi - yj)**2 + (zi - zj)**2)**(3/2)

                axi += axij
                ayi += ayij
                azi += azij

        a.append((axi, ayi, azi))

    return a

We may try using this function to evaluate the gravitational force
between two particles, with with mass $m = 1$, at a distance of $2$.
We expect the "acceleration" be $-1/4$ in the direction of their
seperation.

In [ ]:
m  = [1.0, 1.0]
r0 = [
    (+1.0, 0.0, 0.0),
    (-1.0, 0.0, 0.0),
]
a0ref = [
    (-0.25, 0.0, 0.0),
    (+0.25, 0.0, 0.0),
]

In [ ]:
a0 = acc1(m, r0)

In [ ]:
a0

In [ ]:
assert a0 == a0ref

### Leapfrog Integrator

Our simulation solves Newton's second law of motion, $\mathbf{f} = m
\mathbf{a}$, numerically.
The equation tells us that the acceleration of a body is determined by
the net force acting on it.
In a system where forces are conservative, this law guarantees the
conservation of energy and momentum.
These are critical properties when simulating long-term dynamics.

However, many standard integration methods tend to drift over time,
gradually losing these conservation properties.
The leapfrog algorithm is a symplectic integrator designed to address
this issue.
Its staggered (or "leapfrogging") updates of velocity and position
help preserve energy and momentum much more effectively over extended
simulations.

1. Half-step velocity update (Kick):
   start by updating the velocity by half a time step using the
   current acceleration $a(t)$:
   \begin{align}
     v\left(t+\frac{1}{2}\Delta t\right) = v(t) + \frac{1}{2}\Delta t\, a(t).
   \end{align}

2. Full-step position update (Drift):
   update the position for a full time step using the half-stepped
   velocity:
   \begin{align}
     x(t+\Delta t) = x(t) + \Delta t\, v\left(t+\frac{1}{2}\Delta t\right).
   \end{align}

3. Half-step velocity update (Kick):
   finally, update the velocity by another half time step using the
   new acceleration $a(t+\Delta t)$ computed from the updated
   positions:
   \begin{align}
     v(t+\Delta t) = v\left(t+\frac{1}{2}\Delta t\right) + \frac{1}{2}\Delta t\, a(t+\Delta t).
   \end{align}

Here is a pure Python implementation of the gravitational
acceleration:

In [ ]:
def leapfrog1(m, r0, v0, dt, acc=acc1):

    n  = len(m)

    # vh = v0 + 0.5 * dt * a0
    a0 = acc(m, r0)
    vh = []
    for i in range(n):
        a0xi, a0yi, a0zi = a0[i]
        v0xi, v0yi, v0zi = v0[i]
        vhxi = v0xi + 0.5 * dt * a0xi
        vhyi = v0yi + 0.5 * dt * a0yi
        vhzi = v0zi + 0.5 * dt * a0zi
        vh.append((vhxi, vhyi, vhzi))

    # r1 = r0 + dt * vh
    r1 = []
    for i in range(n):
        vhxi, vhyi, vhzi = vh[i]
        x0i,  y0i,  z0i  = r0[i]
        x1i = x0i + dt * vhxi
        y1i = y0i + dt * vhyi
        z1i = z0i + dt * vhzi
        r1.append((x1i, y1i, z1i))

    # v1 = vh + 0.5 * dt * a1
    a1 = acc(m, r1)
    v1 = []
    for i in range(n):
        a1xi, a1yi, a1zi = a1[i]
        vhxi, vhyi, vhzi = vh[i]
        v1xi = vhxi + 0.5 * dt * a1xi
        v1yi = vhyi + 0.5 * dt * a1yi
        v1zi = vhzi + 0.5 * dt * a1zi
        v1.append((v1xi, v1yi, v1zi))

    return r1, v1

We may try using this function to evolve the two particles under
gravitational force, with some initial velocities.

In [ ]:
v0 = [
    (0.0, +0.5, 0.0),
    (0.0, -0.5, 0.0),
]
v1ref = [
    (-0.024984345739803245, +0.4993750014648409, 0.0),
    (+0.024984345739803245, -0.4993750014648409, 0.0),
]

In [ ]:
r1, v1 = leapfrog1(m, r0, v0, 0.1)

In [ ]:
v1

In [ ]:
assert v1 == v1ref

### Test the Integrator

We test the integrator by evolving the two particle by a time $T =
2\pi$ using $N = 64$ steps.

In [ ]:
from math import pi

N = 64
T = 2 * pi

In [ ]:
dt = T / N
R  = [r0]
V  = [v0]
for _ in range(N):
    r, v = leapfrog1(m, R[-1], V[-1], dt)
    R.append(r)
    V.append(v)

Although our numerical scheme is implemented in pure python, it is
still handy to use `numpy` to slice through the data ...

In [ ]:
import numpy as np

R = np.array(R)
X = R[:,:,0]
Y = R[:,:,1]

and plot the result...

In [ ]:
from matplotlib import pyplot as plt

plt.plot(X, Y, '.-')
plt.gca().set_aspect('equal')

In addition, we may create animation.

In [ ]:
from matplotlib.animation import ArtistAnimation
from IPython.display import HTML
from tqdm import tqdm

def animate(X, Y, ntail=10):
    fig, ax = plt.subplots(1, 1, figsize=(5,5))
    ax.set_xlabel('x')
    ax.set_ylabel('y')

    frames = []
    for i in tqdm(range(len(X))):
        b,e = max(0, i-ntail), i+1
        ax.set_prop_cycle(None)
        f = ax.plot(X[b:e,:], Y[b:e,:], '.-', animated=True)
        frames.append(f)

    anim = ArtistAnimation(fig, frames, interval=50)
    plt.close()
    
    return anim

In [ ]:
anim = animate(X, Y)

HTML(anim.to_html5_video())  # display animation
# anim.save('orbitss.mp4')  # save animation